<a href="https://colab.research.google.com/github/data4class/handwrittendigits/blob/main/Portfolio_Optimization_II_(2025).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import time

# Define stock symbols
stocks = ['RELIANCE.NS', 'INFY.NS', 'ADANIPORTS.NS', 'POWERGRID.NS', 'SBIN.NS'] # Add '^CRSLDX' for index NIFTY 500 data

# Download data for each stock individually
all_prices = []
all_returns = []

for stock in stocks:
        ticker = yf.Ticker(stock)
        hist = ticker.history(period='5y')

        # Extract close prices
        prices = hist['Close'].dropna()

        # Calculate daily returns
        returns = prices.pct_change().dropna()

        all_prices.append(prices.tolist())
        all_returns.append(returns.tolist())

        print(f"Downloaded {stock}: {len(prices)} days, {len(returns)} returns")

        # Small delay to be respectful to the API
        time.sleep(0.1)

# Create DataFrame for easier analysis
returns_df = pd.DataFrame()
prices_df = pd.DataFrame()
for i, stock in enumerate(stocks):
    if i < len(all_returns):
        # Align all return and price series by date
        returns_df[stock] = pd.Series(all_returns[i])
        prices_df[stock] = pd.Series(all_prices[i])


In [ ]:
import numpy as np

# Risk and return of portfolio
def pRisk(w, C):
    return(np.sqrt(w.T*C*w).item())

def pReturn(w, r):
    return(w.T*r)

##Sharpe Ratio

def sharpe(rt, rsk, rf = 0.06):
    return((rt - rf)/rsk)


In [ ]:
C = returns_df.cov()
annual_return = returns_df.sum()/5
C = np.matrix(C)
R = np.matrix(list(annual_return)).reshape(-1, 1)

# Implementating solutions of minimum risk and maximum sharpe ratio problems.
Consider a portfolio of $n$ stocks with covariance matrix $C$ and return vector of individual stock is $R$. Then, \\

Minimum risk problem: $\min_{w, \lambda}\{\frac{1}{2}w^T*C*w + \lambda(w^T*e - 1)\}$ \\
,where $e$ is column vector of $1$'s with size $n$. \\
Solution: $w* = \frac{C^{-1}*e}{e^T*C^{-1}*e}$ \\

Maximum sharpe ratio problem: $\max_{w}\{\frac{w^T*R - r_f}{\sqrt{w^T*C*w}}\}$ \\
,where $r_f$ is risk-free rate. \\
Solution: $w* = \frac{C^{-1}*(R - r_fe)}{e^T*C^{-1}*(R - r_fe)}$ \\


In [ ]:
############### Min risk weight w_min = C^(-1)*e/(e.T*C^(-1)*e)
n = len(stocks)
e = np.ones((n,1))

w_min = np.linalg.inv(C)*e/(e.T*np.linalg.inv(C)*e)
risk_min = pRisk(w_min, C)
return_min = (w_min.T*R).item()


############### Maximum Sharpe ratio weight w_max = C^(-1)*(r - rf*e)/(e.T*C^(-1)*(r - rf*e))
rf = 0.06
e = np.ones((n,1))

w_max = np.linalg.inv(C)*(R - rf*e)/(e.T*np.linalg.inv(C)*(R - rf*e))

risk_max = pRisk(w_max, C)
retrn_max = (w_max.T*R).item()
max_sharpe_ratio = (retrn_max - rf)/risk_max

In [ ]:

##CAPM
# This works when we have index data
# rf = 0.06
# beta = C[0:(n-1),(n-1)]/C[(n-1), (n-1)]
# rm = annual_return[0,(n-1)]
# r = rf + beta*(rm - rf)



#To exclude index data
# n = n-1
# C = C[0:n,0:n]
## Hw: Complete this part

# General portfolio optimization with inequility constrained.

Problem: $\min_{w}\{\frac{1}{2}w^T*C*w + f*w^T \}$, such that $A*w \le b$

In [ ]:
## Quadratic Optimization using CVXOPT
from cvxopt import matrix, solvers
import numpy as np

# Keep original numpy copy
C2 = C.copy()

# Portfolio Optimization:
C = matrix(C)
f = matrix(np.zeros(n))

#Contrains:
# (1) w1 + w2 + ... + wn <= 1
# (2) -w1 - w2 - ... - wn <= -1
# (3) -r1*w1 - r2*w2 - ... - rn*wn <= -rf
# (4) -w1 + 0*w2 + ... + 0*wn <= 0
# (5) 0*w1 - w2 + ... + 0*wn <= 0
# ....
# (n+3) 0*w1 + 0*w2 + ... - wn <= 0
A = np.ones(shape = (n+3, n))
A[1, 0:n] = - A[1, 0:n]
A[2, 0:n] = - R.T
A[3:(n+3), 0:n] = - np.matrix(np.identity(n))

A = matrix(A) # Matrix A in notes
b = matrix([1, -1, -rf] +  np.zeros(n).tolist()) # vector b in notes

Sol = solvers.qp(C, f, A, b)

w = list(Sol['x'])

In [ ]:
print(w)
w = np.matrix(w).reshape(-1, 1)
minrisk = np.sqrt(w.T*C2*w).item()
print(minrisk)

# Balck-litterman model

In [ ]:
print(R,"\n\n", C2)
# Remove the a stock (last one) from the covariance matrix for below example of black-litterman model
C2 = np.delete(np.delete(C2, 4, axis = 0), 4, axis = 1)
n = n-1
R = R[0:n]

In [ ]:
# Black-litterman Model:
# Investors views: (1) r3 >= r1 + r2 --> -r1 -r2 +r3 +0r4 >= 0 (5% pecent uncertainity)
# (2) r4 >= (r1 + r2 + r3)/3 --> -r1 -r2 -r3 +3r4 >= 0 (10% uncertainity)
# (3) r1 >= 0.10 --> r1 0r2 +0r3 +0r4 >= 0.10 (5% uncertainity)
# (4) r4 <= 1 --> 0r1 +0r2 +0r3 -r4 >= -1 (20% uncertainity)

P = np.matrix([[-1, -1, 1, 0],[-1, -1, -1, 3],[1, 0, 0, 0],[0, 0, 0, -1]])
Q = np.matrix([0, 0, 0.1, -1]).T
Omega = np.diag([0.05, 0.10, 0.05, 0.20])
tau = 1 # Please find the find appropriate value of tau from tutorial
C1 = np.linalg.inv(np.linalg.inv(tau*C2) + P.T*np.linalg.inv(Omega)*P )
M = C2 + C1 # Expected covariance

# ER = np.matmul(C1,(np.linalg.inv(tau*C2)*rf + P.T*np.linalg.inv(Omega)*Q)) #Expected returns


# Solve above portfolio optimization using Black-litterman model:
##Replace r with ER and C with M in above optimization problem
C = matrix(M)
f = matrix(np.zeros(n))
A = np.ones(shape = (n+3, n))
A = - A[1, 0:n]

ER = np.array(R).reshape(-1, 1) # Replace this with ER
A[2, 0:n] = - R
A[3:(n+3), 0:n] = - np.matrix(np.identity(n))

A = matrix(A)
b = matrix([1, -1, -rf] +  np.zeros(n).tolist())

Sol = solvers.qp(C, f, A, b)
w = np.matrix(list(Sol['x'])).T
minrisk = np.sqrt(w.T*C*w).item()